# [0] Setup

In [1]:
from pathlib import Path
import zipfile
import os

import xarray as xr
import rioxarray
import zarr

from redplanet.DatasetManager.hash import (
    get_available_algorithms,
    _calculate_hash_from_file,
)

### Inputs

Dataset can be downloaded here: https://astrogeology.usgs.gov/search/map/mars_mgs_mola_dem_463m

See README for more info.

In [2]:
fpath_dem_tif = Path('/home/lain/root/100_work/110_projects/111_mars/raw_data/Mars_MGS_MOLA_DEM_mosaic_global_463m.tif')

assert _calculate_hash_from_file(fpath_dem_tif, 'xxh3_64') == 'a0dc027e687f855f'

---
---
# [1] Load/format given TIF data file to `xarray.DataArray`

Note: native units in the 463m DEM TIF data file are just `x` and `y` (not sure what units), so we reproject using the CRS WKT straight from the 200m DEM TIF file.

In [3]:
known_crs_pre_reprojection = {
    'crs_wkt'     : 'PROJCS["Equirectangular Mars",GEOGCS["GCS_Mars",DATUM["D_Mars",SPHEROID["Mars_localRadius",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
    'spatial_ref' : 'PROJCS["Equirectangular Mars",GEOGCS["GCS_Mars",DATUM["D_Mars",SPHEROID["Mars_localRadius",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
    'GeoTransform': '-10669675.197320545 463.0935415503709 0.0 5334837.598660273 0.0 -463.0935415503709'
}

dat_dem_xr = (
    rioxarray.open_rasterio(
        filename = fpath_dem_tif,
        chunks   = {'x': 'auto', 'y': 'auto'},
    )
)

assert dat_dem_xr.spatial_ref.attrs == known_crs_pre_reprojection

In [4]:
target_crs_wkt = 'GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'

dat_dem_xr = (
    rioxarray.open_rasterio(
        filename = fpath_dem_tif,
        chunks   = {'x': 'auto', 'y': 'auto'},
    )
    .rio.reproject(target_crs_wkt)      ## bottleneck (~45sec)
    .chunk({'x': 'auto', 'y': 'auto'})
    .sel(band=1)
    .drop_vars(['band'])
    .rename({'x': 'lon', 'y': 'lat'})
    .sortby('lat', ascending=True)
    .chunk({'lon': 'auto', 'lat': 'auto'})
    .rename('Mars_MGS_MOLA_DEM_mosaic_global_463m')
)

dat_dem_xr.attrs = {
    'units'   : 'meters',
    'metadata': {
        'source_data': 'https://astrogeology.usgs.gov/search/map/mars_mgs_mola_dem_463m',
        'more_info'  : 'https://github.com/Humboldt-Penguin/redplanet',
    },
}

---
### Inspecting dataset properties

This this point, `dat_dem_xr` looks like this:

![](https://files.catbox.moe/3axrp4.png)

And the CRS is:

In [5]:
known_crs = {
    'crs_wkt'                    : 'GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]',
    'semi_major_axis'            : 3396190.0,
    'semi_minor_axis'            : 3396190.0,
    'inverse_flattening'         : 0.0,
    'reference_ellipsoid_name'   : 'Mars_2000_Sphere_IAU_IAG',
    'longitude_of_prime_meridian': 0.0,
    'prime_meridian_name'        : 'Reference_Meridian',
    'geographic_crs_name'        : 'GCS_Mars_2000_Sphere',
    'horizontal_datum_name'      : 'Mars_2000_(Sphere)',
    'grid_mapping_name'          : 'latitude_longitude',
    'spatial_ref'                : 'GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]',
    'GeoTransform'               : '-180.0 0.007812330461578525 0.0 90.0 0.0 -0.007812060044947162'
}

assert dat_dem_xr.spatial_ref.attrs == known_crs

---
---
# [2] Save to `zarr.ZipStore`

Note: Instead of directly saving dataset as a `zarr.ZipStore`, we:

- Save as `zarr.DirectoryStore`,
- Set all files to have a fixed/consistent timestamp,
- Zip it.

This is functionally equivalent to saving a `zarr.ZipStore`, but standardizing the modtimes ensures the final file always has a consistent hash.

In [6]:
dirpath_out = Path.cwd() / 'output'
dirpath_out.mkdir(exist_ok=True)


## Save as `zarr.DirectoryStore`
fname_dem = 'Mars_MGS_MOLA_DEM_mosaic_global_463m_reproj'
dirpath_dem_zarr = dirpath_out / f'{fname_dem}.zarr'
with zarr.DirectoryStore(dirpath_dem_zarr) as dirstore:
    dat_dem_xr.to_zarr(store=dirstore)


## Normalize modtimes and save as `zarr.ZipStore` (deleting as we go)
fpath_dem_zarrzip = dirpath_out / f'{fname_dem}.zarr.zip'

with zipfile.ZipFile(fpath_dem_zarrzip, "w", compression=zipfile.ZIP_STORED, strict_timestamps=False) as zipf:
    for dirpath_parent, dirnames, fnames in dirpath_dem_zarr.walk(top_down=False):

        for fname in fnames:
            fpath = dirpath_parent / fname
            os.utime(fpath, times=(0,0))
            zipf.write(fpath, arcname = fpath.relative_to(dirpath_dem_zarr))
            fpath.unlink()

        for dirname in dirnames:
            (dirpath_parent / dirname).rmdir()

dirpath_dem_zarr.rmdir()

---
### Final hashes

Mars_MGS_MOLA_DEM_mosaic_global_463m_reproj.zarr.zip
- xxh3_64: 07b987982f52b471
- md5: 82747f05402dcc5b75975b45ceabc47c
- sha256: 9b872030373edd899917c3a49cb72c1dbbb57132f05b0c7b961fccf74bbe0b9b

In [7]:
# print(f'{fpath_dem_zarrzip.name}')
# for alg in get_available_algorithms():
#     print(f'- {alg}: {_calculate_hash_from_file(fpath_dem_zarrzip, alg)}')

In [10]:
assert _calculate_hash_from_file(fpath_dem_zarrzip, 'xxh3_64') == '07b987982f52b471'

---
---
# [Footnote] — Alternative reprojection method to `rio.reproject` with `rasterio` WarpedVRT

Note: when it comes to reprojecting the 463m DEM, here's another code implementation that yields identical results and loads instantly (although I assume speed is lost when calculations are done on the fly). The code is pretty rough (and probably missing some things), but should be fairly easy to adapt if the need arises.

&nbsp;

```python
import rasterio

target_crs_wkt = 'GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'

# vrt = rasterio.vrt.WarpedVRT(rasterio.open(custom_DEM_fpath), crs=target_crs_wkt)
dat_dem_xr_v1 = (
    rioxarray.open_rasterio(
        filename = rasterio.vrt.WarpedVRT(
            src_dataset = rasterio.open(fpath_dem_tif), 
            crs         = target_crs_wkt, 
        ), 
        chunks = {'x': 'auto', 'y': 'auto'}, 
    )
    .sel(band=1).drop_vars(['band', 'spatial_ref'])
    .rename({'x': 'lon', 'y': 'lat'})
    .sortby('lat', ascending=True)
)


dat_dem_xr_v1
```